In [ ]:
# start coding here

In [ ]:
import pypsa
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from datetime import datetime
from cartopy import crs as ccrs
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches

In [ ]:
n = pypsa.Network(snakemake.input.network)

if snakemake.wildcards.interconnect == "texas":
    bus_scale = 1e5
else:
    bus_scale = 5e3

line_scale = 2e3

In [ ]:
# TODO
n.carriers.loc["wind", "color"] = "lightblue"
n.carriers.loc["ng", "color"] = "indianred"

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={"projection": ccrs.PlateCarree()})
n.plot(
    bus_sizes=n.generators.groupby(["bus", "carrier"]).p_nom.sum() / bus_scale,
    bus_alpha=0.7,
    line_widths=n.lines.s_nom / line_scale,
    line_colors="teal",
    ax=ax,
    margin=0.2,
    color_geomap=True,
)

legend_kwargs = {"loc": "upper left", "labelspacing": 2, "frameon": False}
bus_sizes = [1000, 2000, 5000]  # in MW
line_sizes = [2000, 5000]  # in MW
add_legend_circles(
    ax,
    [s / bus_scale for s in bus_sizes],
    [f"{s / 1000} GW" for s in bus_sizes],
    legend_kw={"bbox_to_anchor": (1, 1), **legend_kwargs},
)
add_legend_lines(
    ax,
    [s / line_scale for s in line_sizes],
    [f"{s / 1000} GW" for s in line_sizes],
    legend_kw={"bbox_to_anchor": (1, 0.8), **legend_kwargs},
)
add_legend_patches(
    ax,
    n.carriers.color,
    n.carriers.nice_name,
    legend_kw={"bbox_to_anchor": (1, 0.5), **legend_kwargs},
)
fig.tight_layout()
fig.savefig(snakemake.output.capacities)

In [ ]:
carriers = n.generators.carrier
production = (
    n.generators_t.p.groupby(carriers, axis=1)
    .sum()
    .rename(columns=n.carriers.nice_name)
    / 1e3
)
production = production.loc[:, production.sum() > 0.1]
demand = n.loads_t.p.sum(1).rename("Demand") / 1e3
colors = n.carriers.set_index("nice_name").color[production.columns]

for timeslice in list(range(1, 12)) + ["all"]:
    sns = (
        n.snapshots.get_loc(f"2016-{timeslice}")
        if timeslice != "all"
        else slice(None, None)
    )
    fig, ax = plt.subplots(figsize=(14, 4))
    production[sns].plot.area(ax=ax, color=colors, alpha=0.7, legend="reverse")
    # demand.plot.line(ax=ax, ls='-', color='darkblue')
    ax.legend(bbox_to_anchor=(1, 1), loc="upper left")
    ax.set_ylabel("Power [GW]")
    ax.set_xlabel("")
    fig.tight_layout()
    suffix = (
        "-" + datetime.strptime(str(timeslice), "%m").strftime("%b")
        if timeslice != "all"
        else ""
    )
    path = Path(snakemake.output.operation)
    fig.savefig(path.parent / (path.stem + suffix + path.suffix))